# LangGraph + MCP Agent System with Real LLMs

This notebook demonstrates using real LLMs from different providers with the agent system.

In [1]:
# Enable async support in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Import the agent system and LLM providers
from agent_system import Agent, Tool, AgentSystem, create_python_tool, create_memory_tool
from llm_providers import create_llm, create_llm_for_use_case, RECOMMENDED_MODELS
import asyncio
import json
import os

## Check Available API Keys

In [2]:
# Check which API keys are available
available_providers = []

if os.getenv('OPENAI_API_KEY'):
    available_providers.append('openai')
    print("✓ OpenAI API key found")
else:
    print("✗ OpenAI API key not found")

if os.getenv('ANTHROPIC_API_KEY'):
    available_providers.append('anthropic')
    print("✓ Anthropic API key found")
else:
    print("✗ Anthropic API key not found")

if os.getenv('GOOGLE_API_KEY'):
    available_providers.append('gemini')
    print("✓ Google API key found")
else:
    print("✗ Google API key not found")

# Check if Ollama is running
try:
    import requests
    response = requests.get('http://localhost:11434/api/tags')
    if response.status_code == 200:
        available_providers.append('ollama')
        print("✓ Ollama is running locally")
        models = response.json().get('models', [])
        if models:
            print(f"  Available models: {', '.join([m['name'] for m in models])}")
except:
    print("✗ Ollama not running (start with 'ollama serve')")

✓ OpenAI API key found
✓ Anthropic API key found
✓ Google API key found
✗ Ollama not running (start with 'ollama serve')


## 1. Creating Agents with Different LLM Providers

In [7]:
# Create tools
def calculator_tool(expression: str) -> str:
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

calc_tool = Tool(
    name="calculator",
    func=calculator_tool,
    description="Evaluates mathematical expressions"
)

# Create agents with different LLMs
agents = {}

# OpenAI Agent (if available)
if 'openai' in available_providers:
    agents['openai'] = Agent(
        name="OpenAI_Assistant",
        context={"specialty": "general assistance with OpenAI models"},
        tools=[calc_tool],
        llm_config={
            "provider": "openai",
            "model": "gpt-4-turbo-preview",
            "temperature": 0.7
        }
    )
    print("Created OpenAI agent with GPT-4-turbo")

# Anthropic Agent (if available)
if 'anthropic' in available_providers:
    agents['anthropic'] = Agent(
        name="Claude_Assistant",
        context={"specialty": "thoughtful analysis with Claude"},
        tools=[calc_tool],
        llm_config={
            "provider": "anthropic",
            "model": "claude-3-opus-20240229",
            "temperature": 0.7
        }
    )
    print("Created Anthropic agent with Claude-3-Opus")

# Gemini Agent (if available)
if 'gemini' in available_providers:
    agents['gemini'] = Agent(
        name="Gemini_Assistant",
        context={"specialty": "Google's Gemini model insights"},
        tools=[calc_tool],
        llm_config={
            "provider": "gemini",
            "model": "gemini-1.5-flash",
            "temperature": 0.7
        }
    )
    print("Created Gemini agent")

# Ollama Agent (if available)
if 'ollama' in available_providers:
    agents['ollama'] = Agent(
        name="Local_Assistant",
        context={"specialty": "local Llama model processing"},
        tools=[calc_tool],
        llm_config={
            "provider": "ollama",
            "model": "llama2",  # or "mistral", "codellama", etc.
            "temperature": 0.7
        }
    )
    print("Created Ollama agent with Llama2")

Created OpenAI agent with GPT-4-turbo
Created Anthropic agent with Claude-3-Opus
Created Gemini agent


In [5]:
# try:
#     agents.pop('anthropic')
# except:
#     pass
# try:
#     agents.pop('openai')
# except:
#     pass

In [9]:
# Test the agents with a simple task
test_task = "Please use the calculator tool to compute 15 * 23 + 42"

for provider, agent in agents.items():
    print(f"\n{'='*60}")
    print(f"Testing {provider} agent...")
    print(f"{'='*60}")
    
    result = await agent.run(test_task)
    
    print(f"Agent: {result['agent']}")
    print(f"LLM Provider: {result['llm_provider']}")
    print(f"Tools Used: {result['tools_used']}")
    print(f"\nResponse:\n{result['response']}")


Testing openai agent...
Agent: OpenAI_Assistant
LLM Provider: ChatOpenAI
Tools Used: []

Response:
To compute \(15 \times 23 + 42\), you first multiply 15 by 23, which equals 345. Then, you add 42 to the result of the multiplication.

\[15 \times 23 = 345\]

\[345 + 42 = 387\]

Therefore, \(15 \times 23 + 42 = 387\).

Testing anthropic agent...
Agent: Claude_Assistant
LLM Provider: ChatAnthropic
Tools Used: []

Response:
To calculate 15 * 23 + 42 without using a calculator:

First, multiply 15 by 23:
  23
x 15
----
 115
 230
----
 345

So, 15 * 23 = 345

Then, add 42 to the result:

345 + 42 = 387

Therefore, 15 * 23 + 42 equals 387.

Testing gemini agent...
Agent: Gemini_Assistant
LLM Provider: ChatGoogleGenerativeAI
Tools Used: []

Response:
The calculation 15 * 23 + 42 was not performed using a calculator as requested.  Therefore, I cannot provide a numerical result.  To obtain the answer, the calculation needs to be performed manually or with a calculator.


In [ ]:
# Create a tool that's clearly needed
def large_number_calculator(expression: str) -> str:
    """Handles calculations with very large numbers that LLMs struggle with"""
    try:
        result = eval(expression)
        return f"Exact result: {result:,}"
    except Exception as e:
        return f"Error: {str(e)}"

large_calc_tool = Tool(
    name="large_calculator",
    func=large_number_calculator,
    description="Calculates exact results for large numbers and complex expressions"
)

# Test with a calculation that clearly needs the tool
large_number_task = "Use the large_calculator tool to find the exact value of 123456789 * 987654321"

print("Testing tool usage with large numbers:")
for provider in ['openai', 'gemini']:
    if provider in available_providers:
        agent = Agent(
            name=f"{provider}_calculator",
            context={"instruction": "You must use tools when explicitly asked"},
            tools=[large_calc_tool],
            llm_config={"provider": provider, "temperature": 0.3}
        )
        
        result = await agent.run(large_number_task)
        print(f"\n{provider.upper()}:")
        print(f"  Tools used: {result['tools_used']}")
        print(f"  Response: {result['response'][:200]}...")

## Testing Tool Usage

Let's create a more complex tool to ensure agents actually use tools:

## 2. Creating Agents with Different Models for Different Purposes

In [ ]:
# Show recommended models for different use cases
print("Recommended models by use case:")
print(json.dumps(RECOMMENDED_MODELS, indent=2))

In [ ]:
# Create specialized agents for different purposes
system = AgentSystem()

# Fast agent for simple tasks (using cheaper/faster models)
if 'openai' in available_providers:
    fast_agent = system.create_agent(
        name="FastResponder",
        context={"role": "quick responses for simple queries"},
        llm_config={
            "provider": "openai",
            "model": "gpt-3.5-turbo",  # Faster, cheaper
            "temperature": 0.5
        }
    )
    print("Created fast agent with GPT-3.5-turbo")

# Powerful agent for complex reasoning (using best models)
if 'anthropic' in available_providers:
    powerful_agent = system.create_agent(
        name="DeepThinker",
        context={"role": "complex analysis and reasoning"},
        llm_config={
            "provider": "anthropic",
            "model": "claude-3-opus-20240229",  # Most powerful
            "temperature": 0.7
        }
    )
    print("Created powerful agent with Claude-3-Opus")

# Code specialist (if Ollama with codellama is available)
if 'ollama' in available_providers:
    code_agent = system.create_agent(
        name="CodeExpert",
        context={"role": "code generation and analysis"},
        tools=[create_python_tool()],
        llm_config={
            "provider": "ollama",
            "model": "codellama",  # Specialized for code
            "temperature": 0.3  # Lower temperature for code
        }
    )
    print("Created code agent with CodeLlama")

## 3. Hierarchical Agents with Mixed LLMs

In [ ]:
# Create a team with different LLMs for different roles
if len(available_providers) >= 2:
    # Create worker agents with different LLMs
    workers = []
    
    # Fast worker for quick tasks
    if 'openai' in available_providers:
        fast_worker = system.create_agent(
            name="QuickWorker",
            context={"role": "fast task execution"},
            llm_config={"provider": "openai", "model": "gpt-3.5-turbo"}
        )
        workers.append(fast_worker)
    
    # Analytical worker for complex tasks
    if 'anthropic' in available_providers:
        analytical_worker = system.create_agent(
            name="AnalyticalWorker",
            context={"role": "deep analysis and reasoning"},
            llm_config={"provider": "anthropic", "model": "claude-3-sonnet-20240229"}
        )
        workers.append(analytical_worker)
    
    # Create supervisor with a balanced model
    supervisor = system.create_agent(
        name="TeamSupervisor",
        context={"role": "coordinate team of specialized workers"},
        tools=[worker.as_tool() for worker in workers],
        llm_config={
            "provider": available_providers[0],
            "model": RECOMMENDED_MODELS['balanced'][available_providers[0]]
        }
    )
    
    print(f"Created team with {len(workers)} workers and 1 supervisor")
    print(f"Workers: {[w.name for w in workers]}")
    print(f"Supervisor uses: {supervisor.llm.__class__.__name__}")

## 4. Using create_llm_for_use_case Helper

In [ ]:
# Create agents optimized for different use cases
use_case_agents = {}

# Create agents for each use case with available providers
for use_case in ['fast', 'balanced', 'powerful']:
    for provider in available_providers[:2]:  # Use first 2 available providers
        try:
            llm = create_llm_for_use_case(use_case=use_case, provider=provider)
            agent_name = f"{use_case.capitalize()}_{provider.capitalize()}_Agent"
            
            agent = Agent(
                name=agent_name,
                context={
                    "use_case": use_case,
                    "provider": provider,
                    "optimization": f"Optimized for {use_case} performance"
                },
                llm=llm
            )
            
            use_case_agents[agent_name] = agent
            print(f"Created {agent_name} with {RECOMMENDED_MODELS[use_case][provider]}")
        except Exception as e:
            print(f"Could not create {use_case} agent with {provider}: {e}")

## 5. Comparing LLM Responses

In [ ]:
# Compare how different LLMs respond to the same task
comparison_task = "Explain quantum computing in one paragraph, suitable for a high school student."

print("Comparing responses from different LLMs...\n")

responses = {}
for provider, agent in list(agents.items())[:3]:  # Compare up to 3 providers
    result = await agent.run(comparison_task)
    responses[provider] = result['response']
    
    print(f"\n{'='*60}")
    print(f"{provider.upper()} Response:")
    print(f"{'='*60}")
    print(result['response'])

## 6. Setting Custom API Keys

In [ ]:
# Example of creating an agent with a custom API key (not from environment)
# Uncomment and add your API key to test

# custom_agent = Agent(
#     name="CustomKeyAgent",
#     context={"note": "using custom API key"},
#     llm_config={
#         "provider": "openai",
#         "model": "gpt-4",
#         "api_key": "your-api-key-here"  # Pass API key directly
#     }
# )

## Setup Instructions for Missing Providers

### OpenAI
```bash
export OPENAI_API_KEY="your-key-here"
```

### Anthropic
```bash
export ANTHROPIC_API_KEY="your-key-here"
```

### Google Gemini
```bash
export GOOGLE_API_KEY="your-key-here"
```

### Ollama (Local Llama models)
1. Install Ollama: https://ollama.ai
2. Start Ollama: `ollama serve`
3. Pull a model: `ollama pull llama2` or `ollama pull mistral`

### Available Models by Provider

**OpenAI:**
- gpt-4, gpt-4-turbo-preview
- gpt-3.5-turbo

**Anthropic:**
- claude-3-opus-20240229 (most capable)
- claude-3-sonnet-20240229 (balanced)
- claude-3-haiku-20240307 (fastest)

**Google:**
- gemini-1.5-flash (fast, efficient)
- gemini-1.5-pro (most capable)
- gemini-1.0-pro (previous generation)

**Ollama:**
- llama2 (7b, 13b, 70b)
- mistral
- codellama
- Many more at https://ollama.ai/library

### Note on Gemini
Gemini handles messages differently than other providers - it doesn't support system messages. The agent system automatically detects when you're using Gemini and adapts the message format by combining system instructions into the user message. This is handled transparently, so you can use Gemini agents the same way as others.